classifiers: nn.crossentropyloss = -log-likelihood --- use for logp(y) and -logq(y/x) for support <br>
kl-div: <br>
reconstr-loss: set reduction to none and then take mean of losses per image in the total batch. This gives reconstr-loss per image for further computation<br> 

In [1]:
import sys
import random
import numpy as np
import torch
import learn2learn as l2l
import pandas as pd

from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from PIL.Image import LANCZOS
from config import *

import json
import copy
import math

from torch.distributions.multivariate_normal import MultivariateNormal


In [2]:
torch.cuda.is_available()


True

In [3]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python38.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/datasets-1.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [4]:
#from data.loaders import Omniglotmix, MiniImageNet
from data.taskers import gen_tasks
from src.zoo.archs import CVAE, LVAE
from src.zoo.lpo_utils import proto_distr, classify, set_sets, accuracy, inner_adapt_lpo, setup, kl_div

#from src.utils import Profiler

In [5]:
train_tasks = gen_tasks(dataname='miniimagenet', root='../../mini_imagenet', mode='train', n_ways=5, k_shots=1, q_shots=1, image_transforms=None)

In [5]:
classes = list(range(1623))
random.shuffle(classes)
image_transforms = transforms.Compose([transforms.Resize(28, interpolation=LANCZOS),
                                                    transforms.ToTensor(),
                                                    lambda x: 1.0 - x,
                                                ])
train_tasks = gen_tasks('omniglot', '/home/anuj/Desktop/Work/TU_Delft/research/implement/omniglot', image_transforms=image_transforms, n_ways=5, k_shots=5, q_shots=5, classes=classes[:1100])

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [5]:
train_tasks, valid_tasks, test_tasks, learner, learner, embedder = setup('miniimagenet', '../../mini_imagenet', 5, 3, 7, 5, 3, 7, 'cpu')

In [6]:
task = train_tasks.sample()
device = 'cpu'
n_ways = 5
k_shots = 3
q_shots = 7

In [7]:
data, labels = task
data, labels = data.to(device), labels.to(device)
#data, labels = data.squeeze(0), labels.squeeze(0)
sort = torch.sort(labels)
data = data.squeeze(0)[sort.indices].squeeze(0)
labels = labels.squeeze(0)[sort.indices].squeeze(0)
total = n_ways * (k_shots + q_shots)
queries_index = np.zeros(total)

# Extracting the query datums from the entire task set
for offset in range(n_ways):
    queries_index[np.random.choice(
        k_shots+q_shots, q_shots, replace=False) + ((k_shots + q_shots)*offset)] = True
support = data[np.where(queries_index == 0)]
support_labels = labels[np.where(queries_index == 0)]
queries = data[np.where(queries_index == 1)]
queries_labels = labels[np.where(queries_index == 1)]


In [8]:
queries_labels

tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3,
        3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4])

In [9]:

y_support = F.one_hot(support_labels, num_classes=n_ways)
y_queries = torch.tensor(range(n_ways))
y_queries = y_queries.repeat(n_ways*q_shots)
y_queries = F.one_hot(y_queries, num_classes=n_ways)
qs = queries.repeat_interleave(n_ways, dim=0)

support, qs = embedder(support), embedder(qs)


In [10]:
y_support.shape

torch.Size([15, 5])

In [11]:
support_cap, support_mu, support_log_var = learner(support, y_support)
support_mu

tensor([[-7.5145e-02, -8.9124e-03, -8.2154e-02,  6.1719e-02,  3.6038e-02,
         -7.0984e-03,  1.6753e-02, -2.3661e-02,  1.2746e-01, -1.0573e-01,
          5.3805e-02, -7.6413e-02, -1.4081e-01,  1.0278e-01,  4.3749e-02,
          5.6895e-02,  5.1605e-02, -9.1093e-02, -2.5902e-02,  4.1604e-02,
          3.4948e-02,  8.3711e-02, -5.8687e-02,  5.6211e-02,  4.8789e-02,
          4.1552e-02,  1.2864e-02, -9.5229e-02, -1.5002e-01,  7.9866e-02,
          4.3899e-02,  3.4003e-03, -1.0844e-01,  4.3500e-02, -7.7216e-02,
          2.7521e-02,  8.4472e-02, -2.5079e-05,  5.4244e-02,  6.5318e-02,
          1.4960e-02,  3.0270e-02, -2.9827e-02, -2.4550e-02, -1.8780e-02,
          8.9408e-02, -7.2169e-04, -9.4412e-02,  9.6726e-02, -4.6303e-02,
          6.7923e-02, -1.1995e-01, -7.5040e-03,  8.5708e-02, -9.5887e-04,
          5.6268e-02,  1.6572e-01, -2.9802e-03, -3.8931e-02,  1.5013e-01,
          7.7928e-02, -1.0938e-01, -2.4156e-02, -4.3086e-02],
        [-6.5314e-02, -6.3642e-03, -9.7451e-02,  2

In [12]:
# Building Prototypical distributions
proto_mu, proto_var = proto_distr(
    support_mu, support_log_var, n_ways, k_shots, 'precision_weighted')


In [15]:
proto_var.shape

torch.Size([5, 64])

In [31]:
proto_mu.shape

torch.Size([5, 64])

In [34]:

# Forward pass on the Query datums
queries_cap, queries_mu, queries_log_var = learner(qs, y_queries)
queries_cap

tensor([[ 0.0428,  0.1504, -0.2027,  ...,  0.0693, -0.1145,  0.0263],
        [-0.1058,  0.0255, -0.2681,  ...,  0.0453, -0.1871,  0.0397],
        [-0.0311,  0.1314, -0.1639,  ..., -0.1088, -0.2072,  0.0628],
        ...,
        [ 0.0193,  0.0127, -0.1972,  ...,  0.0714, -0.1562,  0.0559],
        [ 0.0277, -0.0531, -0.3132,  ...,  0.0724, -0.1923, -0.0049],
        [ 0.0644,  0.0406, -0.1644,  ...,  0.0120, -0.1460,  0.0453]],
       grad_fn=<AddmmBackward>)

In [40]:

support_logits = classify(
    mu_p=proto_mu, var_p=proto_var, mu_datums=support_mu)
queries_logits = classify(
    mu_p=proto_mu, var_p=proto_var, mu_datums=queries_mu)
support_logits

tensor([[-23.6021, -23.6018, -23.6142, -23.5726, -23.5880],
        [-23.5873, -23.5628, -23.4816, -23.4948, -23.4916],
        [-23.5813, -23.5522, -23.5197, -23.5076, -23.5279],
        [-23.5949, -23.5359, -23.5181, -23.5067, -23.5110],
        [-23.5810, -23.5382, -23.5119, -23.5014, -23.5085],
        [-23.6005, -23.5409, -23.5303, -23.5203, -23.5227],
        [-23.6427, -23.6106, -23.4794, -23.5159, -23.4984],
        [-23.6293, -23.5917, -23.4693, -23.5036, -23.4954],
        [-23.6215, -23.5772, -23.4727, -23.4979, -23.4884],
        [-23.6039, -23.5669, -23.5206, -23.5002, -23.5154],
        [-23.5838, -23.5446, -23.4925, -23.4868, -23.4953],
        [-23.6467, -23.6168, -23.4848, -23.5051, -23.4993],
        [-23.6246, -23.5816, -23.5140, -23.5177, -23.4837],
        [-23.6001, -23.5622, -23.4982, -23.4963, -23.4792],
        [-23.6877, -23.6445, -23.4968, -23.5423, -23.5072]],
       grad_fn=<SumBackward1>)

In [43]:

# adding up the losses
ce_loss = torch.nn.CrossEntropyLoss(reduction='none')
reconstruction_loss = nn.MSELoss(reduction='none')

In [49]:

L_support = -reconstruction_loss(support_cap, support).view(support.shape[0], -1).mean(dim=1) - ce_loss(
    F.softmax(torch.ones_like(y_support).float(), dim=1), torch.argmax(y_support, dim=1)) - kl_div(support_mu, support_log_var)  # = -L(x_s, y_s)
L_support

tensor([-2.5833, -3.0038, -2.8290, -2.5975, -2.6868, -2.6668, -3.2976, -3.0719,
        -2.9612, -2.6229, -2.8146, -3.3088, -2.8344, -2.8077, -3.4454],
       grad_fn=<SubBackward0>)

In [65]:

L_queries = -reconstruction_loss(queries_cap, qs).view(qs.shape[0], -1).mean(dim=1) - ce_loss(
    F.softmax(torch.ones_like(y_queries).float(), dim=1), torch.argmax(y_queries, dim=1)) - kl_div(queries_mu, queries_log_var)  # = -L(x_q, y_q)
L_queries

tensor([-2.7519, -2.7516, -2.7609, -2.7493, -2.7607, -2.6219, -2.6133, -2.6251,
        -2.6283, -2.6194, -2.7640, -2.7708, -2.7782, -2.7780, -2.7817, -2.7049,
        -2.6945, -2.7047, -2.7047, -2.7028, -2.6130, -2.6090, -2.6101, -2.6148,
        -2.6060, -2.7029, -2.7020, -2.7052, -2.7152, -2.7026, -2.6540, -2.6356,
        -2.6403, -2.6457, -2.6380, -2.5660, -2.5774, -2.5702, -2.5704, -2.5642,
        -2.8104, -2.8126, -2.8134, -2.8128, -2.8140, -2.8794, -2.8642, -2.8698,
        -2.8728, -2.8693, -2.9672, -2.9540, -2.9480, -2.9431, -2.9446, -2.9832,
        -2.9816, -2.9821, -2.9812, -2.9750, -2.5828, -2.5861, -2.5875, -2.5781,
        -2.5856, -2.8765, -2.8712, -2.8730, -2.8725, -2.8709, -2.7327, -2.7147,
        -2.7157, -2.7129, -2.7193, -3.5113, -3.4952, -3.4876, -3.4819, -3.4779,
        -2.5509, -2.5536, -2.5512, -2.5452, -2.5477, -2.7118, -2.7136, -2.7242,
        -2.7072, -2.7143, -3.3958, -3.3839, -3.3937, -3.3829, -3.3937, -2.7552,
        -2.7553, -2.7500, -2.7622, -2.76

In [63]:
yq = torch.argmax(y_queries, dim=1)
yq.view(n_ways*q_shots, n_ways).t()

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]])

In [71]:
L_queries.view(n_ways*q_shots, n_ways).shape

torch.Size([35, 5])

In [76]:
U_queries = torch.mul(F.softmax(queries_logits, dim=1)[
                        ::5, ], L_queries.view(n_ways*q_shots, n_ways)).sum(dim=1) - 0.1*torch.sum(torch.mul(F.softmax(queries_logits, dim=1)[
                            ::5, ], torch.log(F.softmax(queries_logits, dim=1)[
                                ::5, ])), dim=1)
U_queries

tensor([-2.5940, -2.4607, -2.6137, -2.5415, -2.4497, -2.5448, -2.4817, -2.4088,
        -2.6518, -2.7102, -2.7902, -2.8197, -2.4231, -2.7119, -2.5580, -3.3294,
        -2.3888, -2.5534, -3.2292, -2.5962, -3.4405, -2.7984, -3.1919, -2.4627,
        -3.1805, -2.8073, -2.7704, -2.5756, -2.5687, -2.7281, -2.4993, -3.1080,
        -3.1591, -2.5205, -3.0378], grad_fn=<SubBackward0>)

In [79]:
alpha = 0.1*(q_shots/k_shots)
J_alpha = -L_support.mean() - U_queries.mean() + alpha * \
    ce_loss(support_logits, torch.argmax(y_support, dim=1)).mean()
J_alpha

tensor(6.0066, grad_fn=<AddBackward0>)

In [80]:

torch.Tensor([1,2,3,4])**-1

tensor([1.0000, 0.5000, 0.3333, 0.2500])